# Easy Scalping Strategy Part I

In [4]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_15_M_BID_31.01.2019-30.01.2022.csv")
df.tail()

,Local time,Open,High,Low,Close,Volume
105123,30.01.2022 00:45:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0
105124,30.01.2022 01:00:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0
105125,30.01.2022 01:15:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0
105126,30.01.2022 01:30:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0
105127,30.01.2022 01:45:00.000 GMT+0200,1.1145,1.1145,1.1145,1.1145,0.0


In [5]:
df=df[df['Volume']!=0]
df.isna().sum()
df.reset_index(drop=True, inplace=True)
df.tail()

,Local time,Open,High,Low,Close,Volume
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56
74762,28.01.2022 23:45:00.000 GMT+0200,1.11505,1.11506,1.11449,1.11450,1568.78


In [6]:
import pandas_ta as ta
df["EMA50"] = ta.ema(df.Close, length=50)
df["EMA100"] = ta.ema(df.Close, length=100)
df["EMA150"] = ta.ema(df.Close, length=150)

In [7]:
backrollingN = 10
df['slopeEMA50'] = df['EMA50'].diff(periods=1)
df['slopeEMA50'] = df['slopeEMA50'].rolling(window=backrollingN).mean()

df['slopeEMA100'] = df['EMA100'].diff(periods=1)
df['slopeEMA100'] = df['slopeEMA100'].rolling(window=backrollingN).mean()

df['slopeEMA150'] = df['EMA150'].diff(periods=1)
df['slopeEMA150'] = df['slopeEMA150'].rolling(window=backrollingN).mean()

In [8]:
df.tail()

,Local time,Open,High,Low,Close,Volume,EMA50,EMA100,EMA150,slopeEMA50,slopeEMA100,slopeEMA150
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00,1.114836,1.115303,1.116403,-0.000005,-0.000012,-0.000024
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91,1.114832,1.115291,1.116381,-0.000007,-0.000013,-0.000024
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68,1.114827,1.115280,1.116358,-0.000009,-0.000014,-0.000024
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56,1.114835,1.115275,1.116341,-0.000007,-0.000013,-0.000024
74762,28.01.2022 23:45:00.000 GMT+0200,1.11505,1.11506,1.11449,1.11450,1568.78,1.114822,1.115260,1.116317,-0.000007,-0.000013,-0.000023


## EMA Signal

In [10]:
import numpy as np
conditions = [
    ( (df['EMA50']<df['EMA100']) & (df['EMA100']<df['EMA150']) & (df['slopeEMA50']<0) & (df['slopeEMA100']<0) & (df['slopeEMA150']<0) ),
    ( (df['EMA50']>df['EMA100']) & (df['EMA100']>df['EMA150']) & (df['slopeEMA50']>0) & (df['slopeEMA100']>0) & (df['slopeEMA150']>0) )
           ]
choices = [1, 2]
df['EMAsignal'] = np.select(conditions, choices, default=0)

In [11]:
df.tail()

,Local time,Open,High,Low,Close,Volume,EMA50,EMA100,EMA150,slopeEMA50,slopeEMA100,slopeEMA150,EMAsignal
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00,1.114836,1.115303,1.116403,-0.000005,-0.000012,-0.000024,1
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91,1.114832,1.115291,1.116381,-0.000007,-0.000013,-0.000024,1
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68,1.114827,1.115280,1.116358,-0.000009,-0.000014,-0.000024,1
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56,1.114835,1.115275,1.116341,-0.000007,-0.000013,-0.000024,1
74762,28.01.2022 23:45:00.000 GMT+0200,1.11505,1.11506,1.11449,1.11450,1568.78,1.114822,1.115260,1.116317,-0.000007,-0.000013,-0.000023,1


## Total signal

In [12]:
TotSignal = [0] * len(df)
for row in range(0, len(df)):
    TotSignal[row] = 0
    if df.EMAsignal[row]==1 and df.Open[row]>df.EMA50[row] and df.Close[row]<df.EMA50[row]:
        TotSignal[row]=1
    if df.EMAsignal[row]==2 and df.Open[row]<df.EMA50[row] and df.Close[row]>df.EMA50[row]:
        TotSignal[row]=2

df['TotSignal']=TotSignal

In [13]:
df.tail()

,Local time,Open,High,Low,Close,Volume,EMA50,EMA100,EMA150,slopeEMA50,slopeEMA100,slopeEMA150,EMAsignal,TotSignal
74758,28.01.2022 22:45:00.000 GMT+0200,1.11448,1.11503,1.11438,1.11502,457.00,1.114836,1.115303,1.116403,-0.000005,-0.000012,-0.000024,1,0
74759,28.01.2022 23:00:00.000 GMT+0200,1.11502,1.11505,1.11457,1.11474,143.91,1.114832,1.115291,1.116381,-0.000007,-0.000013,-0.000024,1,1
74760,28.01.2022 23:15:00.000 GMT+0200,1.11475,1.11485,1.11460,1.11470,171.68,1.114827,1.115280,1.116358,-0.000009,-0.000014,-0.000024,1,0
74761,28.01.2022 23:30:00.000 GMT+0200,1.11469,1.11513,1.11469,1.11505,228.56,1.114835,1.115275,1.116341,-0.000007,-0.000013,-0.000024,1,0
74762,28.01.2022 23:45:00.000 GMT+0200,1.11505,1.11506,1.11449,1.11450,1568.78,1.114822,1.115260,1.116317,-0.000007,-0.000013,-0.000023,1,1


In [14]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

## Visualizing signals on chart

In [15]:
def pointpos(x):
    if x['TotSignal']==1:
        return x['High']+1e-3
    elif x['TotSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df[20000:22350]
#100:350
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA50, line=dict(color='orange', width=1), name="EMA50"),
                go.Scatter(x=dfpl.index, y=dfpl.EMA100, line=dict(color='blue', width=1), name="EMA100"),
                go.Scatter(x=dfpl.index, y=dfpl.EMA150, line=dict(color='red', width=1), name="EMA150")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [30]:
dfpl

,Local time,Open,High,Low,Close,Volume,EMA50,EMA100,EMA150,slopeEMA50,slopeEMA100,slopeEMA150,EMAsignal,TotSignal,pointpos
20000,20.11.2019 23:45:00.000 GMT+0200,1.10719,1.10727,1.10713,1.10723,206.14,1.106852,1.106912,1.106900,0.000017,0.000006,4.258903e-06,0,0,NaN
20001,21.11.2019 00:00:00.000 GMT+0200,1.10723,1.10741,1.10720,1.10731,315.45,1.106870,1.106920,1.106906,0.000018,0.000007,4.560110e-06,0,0,NaN
20002,21.11.2019 00:15:00.000 GMT+0200,1.10731,1.10751,1.10725,1.10750,345.13,1.106895,1.106931,1.106914,0.000019,0.000008,5.175207e-06,0,0,NaN
20003,21.11.2019 00:30:00.000 GMT+0200,1.10750,1.10759,1.10737,1.10758,353.59,1.106922,1.106944,1.106922,0.000019,0.000008,5.517258e-06,0,0,NaN
20004,21.11.2019 00:45:00.000 GMT+0200,1.10758,1.10780,1.10747,1.10759,254.18,1.106948,1.106957,1.106931,0.000020,0.000009,5.815042e-06,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22345,26.12.2019 00:45:00.000 GMT+0200,1.10913,1.10919,1.10906,1.10908,376.00,1.108495,1.108485,1.108556,0.000004,0.000002,-3.552858e-08,0,0,NaN
22346,26.12.2019 01:00:00.000 GMT+0200,1.10912,1.10916,1.10901,1.10909,782.38,1.108518,1.108497,1.108564,0.000007,0.000003,8.656042e-07,0,0,NaN
22347,26.12.2019 01:15:00.000 GMT+0200,1.10908,1.10929,1.10908,1.10928,308.43,1.108548,1.108513,1.108573,0.000011,0.000005,2.284603e-06,0,0,NaN
22348,26.12.2019 01:30:00.000 GMT+0200,1.10928,1.10953,1.10923,1.10948,634.27,1.108584,1.108532,1.108585,0.000015,0.000007,3.764277e-06,0,0,NaN


## Backtesting Strategy

In [31]:
def SIGNAL():
    return dfpl.TotSignal

In [32]:
from backtesting import Strategy
class MyStrat(Strategy):
    initsize = 0.3
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        
        #if(self.signal1>0 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0):
        #    self.mysize=self.mysize*2
        #elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
        #    self.mysize=self.initsize

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Close[-1] - 40e-4
            tp1 = self.data.Close[-1] + 40e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + 40e-4
            tp1 = self.data.Close[-1] - 40e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

In [33]:
from backtesting import Backtest

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/100, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                 20000.0
End                                   22349.0
Duration                               2349.0
Exposure Time [%]                   65.617021
Equity Final [$]                     81.18201
Equity Peak [$]                     130.01285
Return [%]                          -18.81799
Buy & Hold Return [%]                0.199597
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -41.64033
Avg. Drawdown [%]                   -4.069161
Max. Drawdown Duration                 1271.0
Avg. Drawdown Duration             102.285714
# Trades                                 12.0
Win Rate [%]                        41.666667
Best Trade [%]                       0.363666
Worst Trade [%]                     -0.363768
Avg. Trade [%]                    

In [34]:
bt.plot()

Row(id='1456', ...)